In [119]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [120]:
train = pd.read_csv("train_V2.csv")

In [121]:
train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [122]:
train.shape

(4446966, 29)

In [123]:
train.isnull().any().values

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True], dtype=bool)

In [124]:
train = train.dropna()
train.isnull().any().values

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False], dtype=bool)

In [125]:
# train = train[:1800]
train.shape

(4446965, 29)

A game in PUBG can have up to 100 players fighting each other. But most of the times a game isn't "full". There is no variable that gives us the number of players joined. So lets create one.

In [126]:
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')
train['playersJoined']

0          96
1          91
2          98
3          91
4          97
5          95
6          97
7          96
8          97
9          97
10         99
11         95
12         90
13         97
14         93
15         95
16         98
17         96
18         99
19         98
20         95
21         94
22         97
23         97
24         98
25         93
26         95
27         94
28         95
29         92
           ..
4446936    96
4446937    91
4446938    81
4446939    96
4446940    97
4446941    90
4446942    97
4446943    93
4446944    98
4446945    98
4446946    97
4446947    95
4446948    98
4446949    96
4446950    95
4446951    96
4446952    92
4446953    95
4446954    99
4446955    96
4446956    96
4446957    96
4446958    95
4446959    96
4446960    94
4446961    94
4446962    93
4446963    98
4446964    94
4446965    98
Name: playersJoined, Length: 4446965, dtype: int64

Based on the "playersJoined" feature we can create (or change) a lot of others to normalize their values. For example i will create the "killsNorm" and "damageDealtNorm" features. When there are 100 players in the game it might be easier to find and kill someone, than when there are 90 players. So i will normalize the kills in a way that a kill in 100 players will score 1 (as it is) and in 90 players it will score (100-90)/100 + 1 = 1.1.
And then we scale it from 0 to 1.

In [127]:
train['headshotNorm'] = (train.headshotKills - train.headshotKills.min()) / (train.headshotKills.max() - train.headshotKills.min())*((100-train['playersJoined'])/100 + 1) 
# train.damageDealt.value_counts()
train['damageNorm'] = (train.damageDealt - train.damageDealt.min()) / (train.damageDealt.max() - train.damageDealt.min())*((100-train['playersJoined'])/100 + 1)
train['killsNorm'] = (train.killStreaks - train.killStreaks.min()) / (train.killStreaks.max() - train.killStreaks.min())*((100-train['playersJoined'])/100 + 1)

In [128]:
# train['healsAndBoosts'] = train['heals']+train['boosts']
# train['totalDistance'] = train['walkDistance']+train['rideDistance']+train['swimDistance']
# train.healsAndBoosts = (train.healsAndBoosts - train.healsAndBoosts.min()) / (train.healsAndBoosts.max() - train.healsAndBoosts.min())*((100-train['playersJoined'])/100 + 1)
# train.totalDistance = (train.totalDistance - train.totalDistance.min()) / (train.totalDistance.max() - train.totalDistance.min())*((100-train['playersJoined'])/100 + 1)
# train.head()


In [129]:
# from xgboost import XGBRegressor
# xgb_model = XGBRegressor()
# sample = train.sample(n=50000)
# x = sample.drop(columns=["Id", "groupId", "matchId", "matchType", "winPlacePerc"])
# y = sample["winPlacePerc"]
# xgb_model.fit(x, y)

In [130]:
train['healsAndBoosts'] = train['heals']+train['boosts']
train['totalDistance'] = train['walkDistance']+train['rideDistance']+train['swimDistance']

In [131]:
# Delete Aim Hacks: They will take control of a players aim and automatically target it towards opponents.
train['HeadshotRate'] = train['headshotKills'] / train['kills']
train['AimHack'] = ((train['HeadshotRate'] >= 1) & (train['kills'] >= 10))
train[train['AimHack'] == True].shape

(24, 37)

In [132]:
train.drop(train[train['AimHack'] == True].index, inplace=True)
train.shape

(4446941, 37)

In [133]:
# Speed Hacks: They usually give the player a massive speed increase,
# meaning they can go from one side of the map to the other in seconds.
train['SpeedHack'] = ((train['longestKill'] >= 1000) & (train['kills'] >= 10))
train[train['SpeedHack'] == True].shape

(14, 38)

In [134]:
train['killsWithoutMoving'] = ((train['kills'] > 0) & (train['totalDistance'] == 0))
train[train['killsWithoutMoving'] == True].shape

(1535, 39)

In [135]:
train.drop(train[train['SpeedHack'] == True].index, inplace=True)
train.drop(train[train['killsWithoutMoving'] == True].index, inplace=True)
train.shape

(4445393, 39)

In [136]:
# Delete other Hacks: tempararily only weaponHack here
train[train['weaponsAcquired'] >= 60].shape

(82, 39)

In [137]:
train.drop(train[train['weaponsAcquired'] >= 60].index, inplace=True)
train.shape

(4445311, 39)

In [ ]:
# Delete zombie mode grades